In [ ]:
!pip install -qU plotly
!pip install -q yfinance
!pip install -q riskfolio-lib

# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

from tqdm import tqdm
from scipy.optimize import minimize
import cvxpy as cp

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import seaborn as sns

# Loading Data

In [ ]:
security = pd.read_csv("./dataset/security_info_clean.csv", index_col=[0])
security

In [ ]:
df = pd.read_csv("./dataset/clean_historical_data.csv", index_col=["Date"])
df

In [ ]:
historical_data = df["2005":"2015"].pct_change().fillna(0)
live_data = df["2015":].pct_change().fillna(0)
rf = 0.025 / 252

# Visualization

In [ ]:
fig = px.pie(data_frame=security, names="Sector")
fig.show()

In [ ]:
stats = df["2005":"2015"].pct_change().describe().T.drop(columns=["count"]).reset_index()
stats = stats.rename(columns={"index": "Symbol"})
stats

In [ ]:
fig = px.scatter(data_frame=stats,
             y=stats["mean"] * 252,
             x=stats["std"] * 252 ** 0.5,
             color=security["Market Cap"] / 100_000_000,
             hover_name="Symbol",
             labels={"x": "$\sigma$",
                     "y": "$E(R)$",
                     "color": "Market Cap(M$)"}
        )
fig.add_hline(y=rf * 252)
fig.show()

# Define Back Test Class

In [ ]:
class BackTest():

    def __init__ (self,
                 name,
                 historical_data,
                 live_data,
                 initialize,
                 starting_capital=10_000,
                 rf=0.025 / 252, **kwargs):
        
        self.name = name # Name of the Optimization methods
        self.historical_data = historical_data # Used to find the intilial allocation
        self.live_data = live_data # Used for backtesting
        self.rf = rf # risk free return
        self.starting_capital = starting_capital

        self.daily_return = pd.Series(name=name, dtype=np.float, index=live_data.index) # for portfilio
        self.allocation = initialize(historical_data, **kwargs)
        self.adjust_allocation()

    def run(self, rebalance, k=252, **kwargs):

        # No rebalancing required
        if k == 0:
            start = self.live_data.index[0]
            end = self.live_data.index[-1]

            print("="*50)
            print(f"Period")
            print(f"{start} - {end}")
            self.daily_return = (self.live_data * self.allocation.values).sum(axis=1)
            return

        for n, chunk in (self.live_data).groupby(np.arange(len(self.live_data)) // k):

            start = chunk.index[0]
            end = chunk.index[-1]

            print("="*50)
            print(f"Period #: {n + 1}")
            print(f"{start} - {end}")

            self.show_top_allocation()

            self.daily_return[start:end] = (chunk * self.allocation.values).sum(axis=1)

            self.evalute_preformace(self.daily_return[:end])

            self.allocation = rebalance(self.live_data[:end], **kwargs)# update allocation
            self.adjust_allocation()


    def adjust_allocation(self, n=5):
        print("-"*50)
        print("Allocation Adjustment & Test")
        self.allocation = self.allocation.round(n)
        residual = 1.00 - self.allocation.sum()

        if not np.allclose(residual, 0.00):
            print(f"Weight adjusted with risdual = {residual:0.5f}")
            self.allocation[self.allocation != 0] += residual / len(self.allocation[self.allocation != 0])
        print(f"sum(w) = 1: {np.allclose(self.allocation.sum(), 1.00)}")
        print(f"w >= 0: {(self.allocation >= 0).all()}")
        print("-"*50)
        

    def show_top_allocation(self, n=5):
        print("-"*50)
        print("Top allocations are:")
        for ticker, weight in self.allocation.sort_values(ascending=False).head(n).items():
            print(f"{ticker:<6}: {weight * 100:0.3f}%")
        print("-"*50)


    def evalute_preformace(self, x, return_metric=False):

        metrics = {
            "Absolute Return": x.add(1).prod() - 1,
            "CAGR": x.add(1).prod() ** (252/len(x)) - 1,
            "IR": x.mean() / x.std() * 252 ** 0.5,
            "Volatility":  x.std() * 252 ** 0.5,
            "Max Drawdown": ((x.add(1).cumprod().cummax() - x.add(1).cumprod()) / x.add(1).cumprod().cummax()).max(),
            "Sharpe Ratio": (x.mean() - self.rf) / x.std() * 252 ** 0.5,
            "Sortino Ratio": (x.mean() - self.rf) / x.apply(lambda v: v if v < self.rf else 0).std() * 252 ** 0.5,
        }
        if return_metric:
            return metrics
        print("-"*50)
        print("Metrics: ")
        for k, v in metrics.items():
            if k in ["IR", "Sharpe Ratio", "Sortino Ratio"]:
                print(f"{k:<15}: {v:0.2f}")
            else:
                print(f"{k:<15}: {v * 100:0.2f}%")
        print("-"*50)


    def plot_equity_curve(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.daily_return.index,
                                 y=self.daily_return.add(1).cumprod() * self.starting_capital,
                                 name=f"Equity Curve: {self.name}",
                                 ))

        metrics = self.evalute_preformace(self.daily_return, return_metric=True)

        fig.add_hline(y=self.starting_capital)
        fig.update_layout(dict(xaxis={"title": "Time",
                                      "rangeslider_visible": True,
                                      "rangeselector": {"buttons": [
                                                                    dict(count=1, label="1m", step="month", stepmode="backward"),
                                                                    dict(count=6, label="6m", step="month", stepmode="backward"),
                                                                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                                                                    dict(count=1, label="1y", step="year", stepmode="backward"),
                                                                    dict(step="all")
                                                                    ]
                                                           },
                                        },
                                yaxis={"title": "$"},
                                title={"text": f"Equity Curve of <i>{self.name}</i> Portfilio<br>" +
                                               f"Absolute Return: {metrics['Absolute Return']*100:0.2f}%<br>" +
                                               f"CAGR: {metrics['CAGR']*100:0.2f}%",
                                        "y": 0.95},
                                legend={"orientation": "h",
                                        "yanchor": "bottom",
                                        "y": 1.02,
                                        "xanchor": "right",
                                        "x": 1},
                                showlegend=False
                            ))
        fig.show()
    

    def plot_performance(self,):
        cumreturn = self.daily_return.add(1).cumprod()
        sr = lambda x: (x.mean() - self.rf) / x.std() * 252 ** 0.5

        metrics = self.evalute_preformace(self.daily_return, return_metric=True)
        performance = pd.DataFrame(data={"Cumulative Return": cumreturn,
                                         "Sharpe Ratio": self.daily_return.rolling(window=125).apply(sr),
                                         "Drawdown": -(cumreturn.cummax() - cumreturn) / cumreturn.cummax()})
        
        # Cumulative Return
        fig = px.line(data_frame=performance,
                      y="Cumulative Return",
                      title=f"Cumulative Return: {self.name}<br>" +
                            f"Absolute Return: {metrics['Absolute Return']*100:0.2f}%<br>" +
                            f"CAGR: {metrics['CAGR']*100:0.2f}%<br>",
                      )
        fig.add_hline(y=1.00)
        fig.update_traces(line_color="green")
        fig.show()

        # Drawdown
        fig = px.area(data_frame=performance,
                      y="Drawdown",
                      title=f"Drawdown: {self.name}<br>" +
                            f"Maxdrawn: {metrics['Max Drawdown']*100:0.2f}%",
                      )
        fig.update_traces(line_color="red")
        fig.show()


        # Rolling Sharpe Ratio Over 6-Month
        fig = px.line(data_frame=performance.dropna(),
                      y="Sharpe Ratio",
                      title=f"Rolling Sharpe Ratio: {self.name}<br>" +
                            f"Average Sharpe Ratio: {performance['Sharpe Ratio'].mean():0.2f}",
                      )
        fig.add_hline(y=performance["Sharpe Ratio"].mean(), line_width=3, line_dash="dash")
        fig.update_traces(line_color="red")
        fig.show() 

# Equal Weighted Portfolio

$ w = \frac{1}{N}$ 

In [ ]:
def f(data):
    num_asset = len(data.columns)
    tickers = data.columns
    return pd.Series(data=1/num_asset,
                     index=tickers,
                     name="Equal Weight")
    
equal_weight = BackTest(name="Equal Weight",
                        historical_data=historical_data,
                        live_data=live_data,
                        initialize=f)
equal_weight.run(rebalance=f, k=0)
equal_weight.plot_performance()
#equal_weight.plot_equity_curve()

# Top 5 of Each Sector
$w_i = \frac{1}{K}, \ ∀_i \in T$

In [ ]:
def f(data):
    top_5 = security.groupby("Sector").apply(lambda x: x.sort_values("Market Cap").tail(5))["Symbol"].values
    alloc = pd.Series(data=0, index=df.columns)
    alloc[top_5] = 1 / len(top_5)
    return alloc 

In [ ]:
top_5 = BackTest(name="Top 5 Of Each Sector",
                        historical_data=historical_data,
                        live_data=live_data,
                        initialize=f)
top_5.run(rebalance=f, k=0)
top_5.plot_performance()
#top_5.plot_equity_curve()

# Maximize Return

$ \max\limits_w \mu^Tw$

$st: \\ \sum\limits_i w_i = 1 \\ w_i \ge 0 $
 

In [ ]:
def f(data):
    num_asset = len(data.columns)
    mu = data.mean().values.reshape((num_asset, 1))
    w = cp.Variable(num_asset, nonneg=True) #

    prob = cp.Problem(cp.Maximize(mu.T@w), 
               [cp.sum(w) == 1, 
                w <= 0.10
                ])
    prob.solve(solver=cp.SCS, verbose=True)
    return pd.Series(w.value, index=data.columns, name="Alloc Max Return")

In [ ]:
max_return = BackTest(name="Max Return",
                        historical_data=historical_data,
                        live_data=live_data,
                        initialize=f)
max_return.run(rebalance=f)
max_return.plot_performance()
#max_return.plot_equity_curve()

# Minimize Volatility

$ \min\limits_w w^TΣw$

$ \sum\limits_i w_i = 1\ \\ w_i ≥ 0$ 

In [ ]:
def f(data):
    num_asset = len(data.columns)
    cov = data.cov().values
    w = cp.Variable(num_asset, nonneg=True) #

    prob = cp.Problem(cp.Minimize(cp.quad_form(w, cov)), 
               [cp.sum(w) == 1, 
                w <= 0.3
                ])
    prob.solve(solver=cp.CVXOPT, verbose=True, kktsolver="robust")
    return pd.Series(w.value, index=data.columns, name="Alloc MIN STD")

In [ ]:
min_std = BackTest(name="Min STD",
                        historical_data=historical_data,
                        live_data=live_data,
                        initialize=f)
min_std.run(rebalance=f)
min_std.plot_performance()
#min_std.plot_equity_curve()

# Maxmize Sharpe Ratio

$\max\limits_w \ \frac{\mu^Tw- r_f}{\sqrt{w^TΣw}}$

$ \sum\limits_i w_i = 1\ \\ w_i ≥ 0$


In [ ]:
def f(data, **kwargs):
    rf = kwargs.get("rf", 0.03 / 252)
    num_asset = len(data.columns)
    mu = data.mean().values.reshape((num_asset, 1))
    cov = data.cov().values
    w = cp.Variable(num_asset, nonneg=True) #

    prob = cp.Problem(cp.Maximize((mu.T@w - rf) - (cp.quad_form(w, cov))), 
               [cp.sum(w) == 1, 
                w <= 0.3
                ])
    prob.solve(solver=cp.CVXOPT, verbose=True)
    return pd.Series(w.value, index=data.columns, name="Alloc MAX SR")

In [ ]:
max_sr = BackTest(name="MAX SR",
                        historical_data=historical_data,
                        live_data=live_data,
                        initialize=f, rf=0.025 / 252)
max_sr.run(rebalance=f)
max_sr.plot_performance()
#max_sr.plot_equity_curve()

# General Vanilla Risk Parity Model 

$\min\limits_x \frac{1}{2} xΣx^T - b^T\log{x}$

$x = \frac{w}{\sqrt{wΣw^T}}$

$w \ge 0 \\ \sum\limits_i w_i = 1$





In [ ]:
import riskfolio as rp
def f(data, **kwargs):
    num_asset = len(data.columns)
    # Building the portfolio object
    port = rp.Portfolio(returns=data)  

    b = None # Risk contribution constraints vector

    method_mu = "hist" # Method to estimate expected returns based on historical data.
    method_cov = "hist" # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model = "Classic" # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    rm = "MV" # Risk measure used, this time will be variance
    obj = "MinRisk" # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'

    w = port.rp_optimization(model=model, rm=rm, rf=rf, b=b, hist=hist)

    return pd.Series(w["weights"], index=data.columns, name="Alloc Vanilla Risk Parity")

In [ ]:
risk_parity = BackTest(name="Vanilla Risk Parity",
                        historical_data=historical_data,
                        live_data=live_data,
                        initialize=f, rf=0.025 / 252)
risk_parity.run(rebalance=f)
risk_parity.plot_performance()
#risk_parity.plot_equity_curve()

# K-Means

In [ ]:
from sklearn.cluster import KMeans
mean = historical_data.rolling(52).mean().dropna()
std = historical_data.rolling(52).std().dropna()
X = pd.concat([mean, std], axis=0).T.values
def run_kmean():
    inertia_list = []
    for k in range(2, 15):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(X)
        inertia_list.append(kmeans.inertia_)
                            
    fig = px.line(x=range(2, 15), 
                 y=inertia_list,
              labels={"x": "Number of Cluster",
                      "y": "Inertia"},
              markers=True,
              )
    fig.update_traces(line=dict(width=3))
    fig.show()
run_kmean()

In [ ]:
def plot_clusters(data, k):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    k_labels = kmeans.labels_

    fig = px.scatter(x=historical_data.std() * 252 ** 0.5,
                     y=historical_data.mean() * 252,
                     hover_name=historical_data.columns,
                     color=[str(x) for x in k_labels],
                     labels={"x": "Variance", "y": "Return",
                             "color": "K"}
                     )
    fig.show()
plot_clusters(historical_data, k=3)

In [ ]:
def f(data, k, which_k=-1):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    labels = kmeans.labels_
    alloc = pd.Series(data=0, index=df.columns, name="Alloc K-Mean")
    temp = pd.DataFrame(data={"Symbol": security["Symbol"].values,
                            "k": [str(x) for x in labels],
                            "Market Cap": security["Market Cap"].values,
                            "Sharpe Ratio": stats["mean"] / stats["std"]
                            })
    if which_k != -1:
        ids_k = temp[temp["k"]==k]["Symbol"]
        alloc[ids_k] = 1
        allo = alloc.mean()

    top_5 = temp.groupby("k").apply(lambda x: x.sort_values("Sharpe Ratio").tail(5))["Symbol"].values
    alloc[top_5] = 1 / len(top_5)
    return alloc

In [ ]:
kmean_div = BackTest(name="K-Mean",
                        historical_data=historical_data,
                        live_data=live_data,
                        initialize=f, rf=0.025 / 252, k=3)
kmean_div.run(rebalance=f, k=0)
kmean_div.plot_performance()

# Result Summary

In [ ]:
def plot_all_eqauity_curve(starting_capital=10_000):
    equity = pd.DataFrame(data={
        "Equal Weight": equal_weight.daily_return,
        "Top 5 Of Each Sector": top_5.daily_return,
        "Max Return": max_return.daily_return,
        "Max SR": max_sr.daily_return,
        "Min STD": min_std.daily_return,
        "Vanilla Risk Parity": risk_parity.daily_return,
        "K-Mean": kmean_div.daily_return,
    }).apply(lambda col: col.add(1).cumprod() * starting_capital)

    fig = px.line(data_frame=equity, title="Equity Curve", labels={"value": "$",
                                                                   "variable": ""})
    fig.add_hline(y=starting_capital, )
    fig.show()
plot_all_eqauity_curve()

In [ ]:
def plot_drawdown():
    drawdown = pd.DataFrame(data={
        "Equal Weight": equal_weight.daily_return,
        "Top 5 Of Each Sector": top_5.daily_return,
        "Max Return": max_return.daily_return,
        "Max SR": max_sr.daily_return,
        "Min STD": min_std.daily_return,
        "Vanilla Risk Parity": risk_parity.daily_return,
        "K-Mean": kmean_div.daily_return,
    }).apply(lambda col: (col.add(1).cumprod() - col.add(1).cumprod().cummax()) / col.add(1).cumprod().cummax())
    fig = px.area(data_frame=drawdown, title="Drawdown", labels={"value": "%",
                                                                "variable": ""})
    fig.show()
plot_drawdown()

In [ ]:
def all_metrics(rf):
    port_lst = [ equal_weight, 
                top_5,
                max_return,
                max_sr,
                min_std,
                risk_parity,
                kmean_div]

    metrics = pd.DataFrame(index=["Absolute Return",
                                  "CAGR",
                                  "IR",
                                  "Volatility",
                                  "Max Drawdown",
                                  "Sharpe Ratio",
                                  "Sortino Ratio"],
                           columns=[x.name for x in port_lst])
    for x, name in zip(port_lst, metrics.columns):
        x = x.daily_return
        metric = {
            "Absolute Return": x.add(1).prod() - 1,
            "CAGR": x.add(1).prod() ** (252/len(x)) - 1,
            "IR": x.mean() / x.std() * 252 ** 0.5,
            "Volatility":  x.std() * 252 ** 0.5,
            "Max Drawdown": ((x.add(1).cumprod().cummax() - x.add(1).cumprod()) / x.add(1).cumprod().cummax()).max(),
            "Sharpe Ratio": (x.mean() - rf) / x.std() * 252 ** 0.5,
            "Sortino Ratio": (x.mean() - rf) / x.apply(lambda v: v if v < rf else 0).std() * 252 ** 0.5,
        }

        metrics.loc[:, name] = metric.values()
    
    return metrics

all_metrics(rf)


In [ ]:
def compare(lst, starting_capital=10_000, rf=0.025/252):
    equity = pd.DataFrame()
    drawdown = pd.DataFrame() 

    for x in lst:
        equity[x.name] = x.daily_return
        drawdown[x.name] = x.daily_return

    equity = equity.apply(lambda col: col.add(1).cumprod() * starting_capital)
    drawdown = drawdown.apply(lambda col: (col.add(1).cumprod() - col.add(1).cumprod().cummax()) / col.add(1).cumprod().cummax())
    
    metrics = all_metrics(rf=rf)

    title = "Equity Curve<br>"

    for x in lst:
        name = x.name
        abs_ret_v = metrics.at["Absolute Return", name]
        cagr_v = metrics.at["CAGR", name]
        std_v = metrics.at["Volatility", name]
        sr_v = metrics.at["Sharpe Ratio", name]
        title += "" + \
                f"<i>{name:<13}</i>: " + \
                f"Absolute Return={abs_ret_v * 100:0.2f}%, " + \
                f"CAGR={cagr_v * 100:0.2f}%, " + \
                f"STD={std_v * 100:0.2f}%, " + \
                f"SR={sr_v:0.2f}"+ "<br>"


    fig = px.line(data_frame=equity,
                  title=title, labels={"value": "$",
                                       "variable": ""})
    fig.update_layout(margin=dict(l=50, r=50, t=100, b=50), title={"y": 0.95})
    fig.add_hline(y=starting_capital)
    fig.show()

    print()

    title = "Drawdown<br>"
    for x in lst:
        name = x.name
        max_d = metrics.at["Max Drawdown", name]
        title += "" + \
                f"<i>{name:<13}</i>: " + \
                f"Max Drawdown={max_d * 100:0.2f}%, " + \
                 "<br>"  

    fig = px.area(data_frame=drawdown, title=title, labels={"value": "%",
                                                            "variable": ""})
    fig.update_layout(margin=dict(l=50, r=50, t=100, b=50), title={"y": 0.95})
    fig.show()

In [ ]:
compare([equal_weight, top_5])

In [ ]:
compare([max_return, min_std, max_sr])

In [ ]:
compare([kmean_div, risk_parity])

In [ ]:
all_metrics(rf=0.025/252).to_csv("./dataset/metrics.csv")